# Imports

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
import numpy as np
import matplotlib.pyplot as plt

# Read Data

In [ ]:
# This cell reads in parameters and response data from Excel files and combines them into a single dataframe
# Check cell outputs to make sure everything looks good

# parameters_file = "Multi-Threshold Analysis Data" # Excel file to pull parameters from
# parameters_sheet = "Suzuki Yields and Parameters" # Sheet in the Excel file to pull parameters from
# parameters_start_col = 2   # 0-indexed column number where the parameters start
# parameters_num_parameters = 190 # Number of parameters in the parameters file
# parameters_num_responses = 450 # Number of responses/ligands in the parameters file
# parameters_y_label_col = 0  # 0-indexed column number where the ligand labels are
# parameters_header_rows = 0 # Number of rows to skip when reading the parameters

# response_file = "Multi-Threshold Analysis Data" # Excel file to pull responses from
# response_sheet = "Suzuki Yields and Parameters" # Sheet in the Excel file to pull responses from
# response_num_samples = 450 # Number of samples/reactions in the response file
# response_col = 1 # 0-indexed column number for the responses
# response_y_label_col = 0  # 0-indexed column number where the ligand labels are
# response_header_rows = 1 # Number of rows to skip when reading the responses

# RESPONSE_LABEL = "Yield (%)" # How you want to refer to your response variable in the dataframe

parameters_file = "8_16_combined_basic_parameters" # Excel file to pull parameters from
parameters_sheet = "Sheet1" # Sheet in the Excel file to pull parameters from
parameters_start_col = 2   # 0-indexed column number where the parameters start
parameters_num_parameters = 3247 # Number of parameters in the parameters file
parameters_num_responses = 192 # Number of responses/ligands in the parameters file
parameters_y_label_col = 0  # 0-indexed column number where the ligand labels are
parameters_header_rows = 0 # Number of rows to skip when reading the parameters

response_file = "8_16_combined_basic_parameters" # Excel file to pull responses from
response_sheet = "Sheet1" # Sheet in the Excel file to pull responses from
response_num_samples = 450 # Number of samples/reactions in the response file
response_col = 1 # 0-indexed column number for the responses
response_y_label_col = 0  # 0-indexed column number where the ligand labels are
response_header_rows = 0 # Number of rows to skip when reading the responses

RESPONSE_LABEL = "Pdt/Hydro" # How you want to refer to your response variable in the dataframe

# --------------------------------------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE
# --------------------------------------------------------------------------------------------------------------

# Actually start reading stuff into dataframes
parameters_df = pd.read_excel("./InputData/" + parameters_file + ".xlsx",
                              parameters_sheet,
                              header = parameters_header_rows,
                              index_col = parameters_y_label_col,
                              nrows = parameters_num_responses + 1,
                              usecols = list(range(0, (parameters_num_parameters + parameters_start_col)))
                              )
response_df = pd.read_excel("./InputData/" + response_file + ".xlsx",
                            response_sheet,
                            header = response_header_rows,
                            index_col = response_y_label_col,
                            nrows = response_num_samples,
                            usecols = list(range(0, response_col + 1))
                            )


# Drop any columns before parameters_start_col that are not the index column
parameters_columns_to_keep = [col for col in range(0, len(parameters_df.columns)) if col >= parameters_start_col-1]
parameters_df = parameters_df.iloc[:,parameters_columns_to_keep]

# Combine the two dataframes into the master dataframe
response_df.drop(response_df.columns[0:response_col-1], axis = 'columns', inplace = True)
data_df = response_df.merge(parameters_df, left_index = True, right_index = True)
data_df.rename(columns = {data_df.columns.values[0]: RESPONSE_LABEL}, inplace = True) # Converts the output column name from whatever it is on the spreadsheet
data_df.dropna(inplace = True) # Remove any rows with blanks

# This converts all the data to numeric values since it was reading them in as non-numeric objects for some reason
for column in data_df.columns:
    data_df[column] = pd.to_numeric(data_df[column], errors='coerce')

# Print out the data distribution of the response variable
plt.figure(figsize=(5, 5))
plt.hist(data_df[RESPONSE_LABEL], color='grey')
plt.xlabel(RESPONSE_LABEL, fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.show()

# Display the dataframe
display(data_df)

# Cross Terms

Pick a cell below to run depending on which cross terms you want, then save/export them with the final cell

## Full Cross-Terms Calculation

In [ ]:
# This cell appends all possible cross terms to the main dataframe

new_columns = {}
parameter_columns = data_df.drop(columns=[RESPONSE_LABEL]).columns

for parameter1 in parameter_columns:
    for parameter2 in parameter_columns:
        if parameter1 != parameter2:
            crossterm = parameter1 + " * " + parameter2
            new_columns[crossterm] = data_df[parameter1] * data_df[parameter2]

print(f'Number of cross terms: {len(new_columns)}')

cross_terms_df = pd.DataFrame(new_columns)
cross_terms_df = pd.concat([data_df, cross_terms_df], axis=1)

display(cross_terms_df)

## Limitted Cros-Terms Calculation

In [ ]:
# This cell appends all cross terms made from a subset of parameters to the main dataframe

parameters_to_cross = range(1, 50)
# parameters_to_cross = list(range(0, 10)) + list(range(20, 30)) # Example of how to cross a non-continuous subset of parameters

# --------------------------------------------------------------------------------------------------------------

parameter_columns = data_df.columns[parameters_to_cross]
print(f'Selected parameters: {list(parameter_columns)}')

new_columns = {}

for parameter1 in parameter_columns:
    for parameter2 in parameter_columns:
        if parameter1 != parameter2:
            crossterm = parameter1 + " * " + parameter2
            new_columns[crossterm] = data_df[parameter1] * data_df[parameter2]

print(f'Number of cross terms: {len(new_columns)}')

cross_terms_df = pd.DataFrame(new_columns)
cross_terms_df = pd.concat([data_df, cross_terms_df], axis=1)

display(cross_terms_df)

## Cross-Terms Between Ranges

In [ ]:
# This cell appends all cross terms made by crossing two subsets of parameters to the main dataframe

parameters_to_cross_1 = range(1, 10)
parameters_to_cross_2 = range(21, 30)

# --------------------------------------------------------------------------------------------------------------

parameter_columns_1 = data_df.columns[parameters_to_cross_1]
parameter_columns_2 = data_df.columns[parameters_to_cross_2]
print(f'First selected parameter set: {list(parameter_columns_1)}')
print(f'Second selected parameter set: {list(parameter_columns_2)}')

new_columns = {}

for parameter1 in parameter_columns_1:
    for parameter2 in parameter_columns_2:
        if parameter1 != parameter2:
            crossterm = parameter1 + " * " + parameter2
            new_columns[crossterm] = data_df[parameter1] * data_df[parameter2]

print(f'Number of cross terms: {len(new_columns)}')

cross_terms_df = pd.DataFrame(new_columns)
cross_terms_df = pd.concat([data_df, cross_terms_df], axis=1)

display(cross_terms_df)

## Export Cross-Terms Dataframe

In [ ]:
# This will export the dataframe to an Excel file in the InputData folder
# It will add a blank row between the column names and the data due to how it handles the MultiIndex, so you may want to delete that row before using the data

export_filename = 'cross_terms.xlsx'

# --------------------------------------------------------------------------------------------------------------

cross_terms_df.to_excel(f'OutputData/{export_filename}')

# Boruta Feature Selection

In [ ]:
# Initialize and fit the Boruta feature selector
# Basically all of the arguments here are default so you might play around with them to suit your needs
# Once you get up into the 1000s of features, it might take a while to run, especially if you're using the full cross-term dataset
# In the final itteration, the number of rejected features is reported incorrectly but everything is still running properly (according to the BorutaPy documentation)

# Only use one of the following lines
boruta_df = data_df.copy() # If you want to use the original data
# boruta_df = cross_terms_df.copy() # If you want to use the data with cross terms

# Set details of the random forest regressor to be used in the Boruta feature selector
rf = RandomForestRegressor(n_jobs=-1, max_depth=5)

# {perc} is the percentage of the maximum importance of the shadow features to set as the cutoff
    # Lower {perc} means you'll get more false positive features kept, but are less likely to discard relevant features
# {max_iter} is the maximum number of iterations to run before stopping
    # Let it go longer if you want to more fully resolve the tentative features
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1, perc=75, max_iter=100) 

# --------------------------------------------------------------------------------------------------------------
# EDIT ABOVE THIS LINE
# --------------------------------------------------------------------------------------------------------------

# Run the Boruta feature selector
parameter_columns = boruta_df.drop(columns=[RESPONSE_LABEL]).columns
feat_selector.fit(np.asarray(boruta_df[parameter_columns]), np.asarray(boruta_df[RESPONSE_LABEL]))

# Compile the names of confirmed and tentatively accepted features into a list
parameters_confirmed = [parameter_columns[i] for i in range(0,len(feat_selector.support_)) if feat_selector.support_[i]]
parameters_tentative = [parameter_columns[i] for i in range(0,len(feat_selector.support_)) if feat_selector.support_weak_[i]]
parameters_confirmed.extend(parameters_tentative)

# Build the final dataframe with the output and only the confirmed and tentatively accepted features
columns_to_keep = [RESPONSE_LABEL] + [column for column in boruta_df.columns if column in parameters_confirmed]
boruta_df_output = boruta_df[columns_to_keep]
display(boruta_df_output)

## Export Boruta Dataframe

In [ ]:
# This will export the dataframe to an Excel file in the InputData folder
# It will add a blank row between the column names and the data due to how it handles the MultiIndex, so you may want to delete that row before using the data

export_filename = 'boruta_parameters.xlsx'

# --------------------------------------------------------------------------------------------------------------
display(boruta_df_output)

boruta_df_output.to_excel(f'OutputData/{export_filename}')